In [0]:
!pip install syft

In [0]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)

# Create virtual worker

In [0]:
bob = sy.VirtualWorker(hook, id='bob')

In [0]:
bob._objects

{}

In [0]:
x = th.Tensor([1, 2, 3, 4, 5])

In [0]:
x = x.send(bob)

In [0]:
bob._objects

{83921711121: tensor([1., 2., 3., 4., 5.])}

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.location == bob

True

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:4129042945 -> bob:83921711121]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.])

In [0]:
bob._objects

{}

# Create another worker

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.Tensor(range(10))
x

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [0]:
x2 = x.send(bob, alice)

In [0]:
x2

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:7953421722 -> bob:19177016834]
	-> [PointerTensor | me:52492661055 -> alice:44069265080]

In [0]:
x2.child

MultiPointerTensor>{'bob': [PointerTensor | me:7953421722 -> bob:19177016834], 'alice': [PointerTensor | me:52492661055 -> alice:44069265080]}

In [0]:
x2.child.child

{'alice': [PointerTensor | me:52492661055 -> alice:44069265080],
 'bob': [PointerTensor | me:7953421722 -> bob:19177016834]}

In [0]:
type(x)

syft.frameworks.torch.tensors.interpreters.native.Tensor

In [0]:
x2.get()

[tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]),
 tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])]

# Some operation

In [0]:
bob.clear_objects
alice.clear_objects

<bound method ObjectStorage.clear_objects of <VirtualWorker id:alice #objects:0>>

In [0]:
x = th.Tensor(range(10)).send(bob)
y = th.Tensor(range(10)).send(bob)

In [0]:
z = x + y
z

(Wrapper)>[PointerTensor | me:35016223044 -> bob:94844115513]

In [0]:
z.get()

tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:80359743887 -> bob:16635224020]

In [0]:
x.get()

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
y.get()

tensor([1., 1., 1., 1., 1.], requires_grad=True)

In [0]:
z.get()

tensor(20., requires_grad=True)

# Garbage collection

In [0]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [0]:
bob._objects

{}

In [0]:
x = th.Tensor(range(10)).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:54651829605 -> bob:89765469781]

In [0]:
bob._objects

{89765469781: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])}

In [0]:
x.child.garbage_collect_data

True

In [0]:
x = "Shudipto"

In [0]:
bob._objects

{89765469781: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])}

In [0]:
del x

In [0]:
bob._objects

{89765469781: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])}

# Simple linear MOdel

In [0]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [0]:
x = th.tensor([[1.0,1.0],[1.0,0.], [0.,1.], [0.,0.]], requires_grad=True).send(bob)
x

(Wrapper)>[PointerTensor | me:87670850020 -> bob:89582183293]

In [0]:
y = th.tensor([[1.0,1.0],[1.0,0.], [0.,1.], [0.,0.]], requires_grad=True).send(bob)
y

(Wrapper)>[PointerTensor | me:56678110645 -> bob:18602992258]

In [0]:
weigths = th.tensor([[0.,], [0.,]], requires_grad=True).send(bob)
weigths

(Wrapper)>[PointerTensor | me:47666451614 -> bob:79786557435]

In [0]:
bob._objects

{18602992258: tensor([[1., 1.],
         [1., 0.],
         [0., 1.],
         [0., 0.]], requires_grad=True), 79786557435: tensor([[0.],
         [0.]], requires_grad=True), 89582183293: tensor([[1., 1.],
         [1., 0.],
         [0., 1.],
         [0., 0.]], requires_grad=True)}

In [0]:
for i in range(10):
  pred = x.mm(weigths)
  loss = ((pred - y)**2).sum()
  loss.backward()
  weigths.data.sub_(weigths.grad * 0.01)
  weigths.grad *= 0
  print(loss.get().data)

tensor(4.)
tensor(3.3232)
tensor(2.7991)
tensor(2.3932)
tensor(2.0789)
tensor(1.8355)
tensor(1.6470)
tensor(1.5010)
tensor(1.3880)
tensor(1.3005)


# Toy Example

In [0]:
from torch import nn, optim

data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

model = nn.Linear(2,1)

opt = optim.SGD(params=model.parameters(), lr=0.1)

def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.1032)
tensor(0.6460)
tensor(0.3696)
tensor(0.2400)
tensor(0.1590)
tensor(0.1061)
tensor(0.0713)
tensor(0.0482)
tensor(0.0328)
tensor(0.0225)
tensor(0.0155)
tensor(0.0108)
tensor(0.0076)
tensor(0.0054)
tensor(0.0038)
tensor(0.0028)
tensor(0.0020)
tensor(0.0015)
tensor(0.0011)
tensor(0.0008)


## Now use federate learning

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [121]:
train()

tensor(6.8638, requires_grad=True)
tensor(0.1173, requires_grad=True)
tensor(0.1657, requires_grad=True)
tensor(0.1406, requires_grad=True)
tensor(0.0525, requires_grad=True)
tensor(0.0902, requires_grad=True)
tensor(0.0302, requires_grad=True)
tensor(0.0564, requires_grad=True)
tensor(0.0185, requires_grad=True)
tensor(0.0356, requires_grad=True)
tensor(0.0116, requires_grad=True)
tensor(0.0227, requires_grad=True)
tensor(0.0074, requires_grad=True)
tensor(0.0146, requires_grad=True)
tensor(0.0048, requires_grad=True)
tensor(0.0095, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0063, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0042, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0029, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0004, requi